# 데이터 정제 및 준비
* 데이터 분석과 모델링 작업은 데이터 준비 과정(데이터 불러오기/정제/변형/재정렬)에 시간을 많이 들이게 됨
* 파이썬 표준 라이브러리를 Pandas와 함께 사용하면 원하는 형태로 가공 가능
* pandas는 유연하고 빠른 고수준의 알고리즘과 처리 기능 제공

## 결측치, 중복데이터, 문자열 처리 그리고 다른 분석적 데이터 변환에 대한 도구들
### Part1. 누락된 데이터 처리
* pandas의 설계 목표: 누락 데이터를 가능한 한 쉽게 처리하기
* 산술 데이터에 한해 pandas는 누락 데이터를 실숫값인 NaN으로 취급
  * 누락된 값을 쉽게 찾을 수 있도록 하는 파수병 역할

In [1]:
import pandas as pd
import numpy as np
from numpy import nan as NA

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
# 결측치를 NA로 취급하는 개념은 R 프로그래밍 언어에서 차용
# 분석 애플리케이션에서 NA 데이터는 데이터가 존재하지 않거나, 존재하더라도 데이터 수집 과정에서 검출되지 않았음을 의미
# 데이터 정제 과정에서 결측치를 "수집 과정에서의 실수" 혹은 "결측치로 인한 잠재적인 편향"을 찾아내는 수단으로 인식할 것!
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
# 파이썬의 내장 None 값 또한 NA 값으로 취급
string_data[0] = None

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### MARK: 누락된 데이터 골라내기
1. pandas.isnull이나 불리언 색인을 사용해 직접 손으로 제거하기
2. dropna 메서드 활용하기
  * Series에 dropna 메서드를 적용하면 널이 아닌 데이터와 색인값만 들어 있는 Series 반환

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
# 위 코드는 다음과 동일
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [11]:
# DataFrame의 경우 비교적 복잡
# 모두 NA 값인 로우나 컬럼을 제외 or NA값을 하나라도 포함하고 있는 로우나 컬럼  제외
# dropna는 기본적으로 NA값을 하나라도 포함하는 로우 제외
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])

In [12]:
# NA가 하나라도 있는 1, 2, 3행 모두 제거
cleaned = data.dropna()

In [13]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [14]:
# how='all' 옵션을 넘기면 모두 NA 값인 로우만 제외
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [15]:
# 컬럼을 제외시키는 방법도 동일하게 동작. 단, axis=1 옵션을 줄 것!
data[4] = NA

In [16]:
data[4]

0   NaN
1   NaN
2   NaN
3   NaN
Name: 4, dtype: float64

In [17]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
# how='all' 옵션 덕분에 모든 셀이 NA인 4열만 지워진 DataFrame abs반환
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
# how='all' 옵션이 없다면 모든 셀이 지워질 것,,, (NA 값이 하나도 없는 열이 없기 땜운!)
data.dropna(axis=1)

""
0
1
2
3


In [20]:
# DataFrame의 로우를 제외시키는 방법은 시계열 데이터에 주로 사용되는 경향이 있음 (먼 소리야)
# 몇 개 이상의 값이 들어있는 로우만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 됨
df = pd.DataFrame(np.random.randn(7, 3))

In [21]:
df

,0,1,2
0,-0.720023,-0.490937,-0.930046
1,-0.339209,-1.218822,0.182771
2,-2.205510,1.556723,-1.255686
3,1.715908,1.722991,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [22]:
df.iloc[:4, 1] = NA

In [23]:
df

,0,1,2
0,-0.720023,NaN,-0.930046
1,-0.339209,NaN,0.182771
2,-2.205510,NaN,-1.255686
3,1.715908,NaN,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [24]:
df.iloc[:2, 2] = NA

In [25]:
df

,0,1,2
0,-0.720023,NaN,NaN
1,-0.339209,NaN,NaN
2,-2.205510,NaN,-1.255686
3,1.715908,NaN,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [26]:
# NA 값이 하나도 없는 행만 반환
df.dropna()

,0,1,2
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [27]:
df.dropna(thresh=2) # 즉, 한 행에 값이 2개 이상인 경우!

,0,1,2
2,-2.205510,NaN,-1.255686
3,1.715908,NaN,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [29]:
df.dropna(axis=1, thresh=4) # axis=1 : 열, 한 열에 값이 4개 이상인 경우

,0,2
0,-0.720023,NaN
1,-0.339209,NaN
2,-2.205510,-1.255686
3,1.715908,-0.009290
4,-0.676623,0.578823
5,0.954759,1.187622
6,1.168148,-0.392240


### MARK: 결측치 채우기
* 누락된 값을 제외시키지 않고 데이터 상의 '구멍'을 메우고 싶은 경우 fillna 메서드 활용
  * 누락된 값을 제외시키면, 잠재적인 다른 데이터도 버려질 가능성 존재
* fillna 메서드에 채워넣고 싶은 값 넘겨주기

In [30]:
df

,0,1,2
0,-0.720023,NaN,NaN
1,-0.339209,NaN,NaN
2,-2.205510,NaN,-1.255686
3,1.715908,NaN,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [32]:
df.fillna(0)

,0,1,2
0,-0.720023,0.000000,0.000000
1,-0.339209,0.000000,0.000000
2,-2.205510,0.000000,-1.255686
3,1.715908,0.000000,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [33]:
# fillna에 사전값을 넘겨서 각 컬럼마다 다른 값을 채울 수도 있음
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.720023,0.500000,0.000000
1,-0.339209,0.500000,0.000000
2,-2.205510,0.500000,-1.255686
3,1.715908,0.500000,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [34]:
# fillna는 새로운 객체를 반환하지만 기존 객체를 변경할 수도 있음
_ = df.fillna(0, inplace=True)

In [35]:
df

,0,1,2
0,-0.720023,0.000000,0.000000
1,-0.339209,0.000000,0.000000
2,-2.205510,0.000000,-1.255686
3,1.715908,0.000000,-0.009290
4,-0.676623,0.982253,0.578823
5,0.954759,0.155706,1.187622
6,1.168148,0.392956,-0.392240


In [36]:
# 재색인에서 사용 가능한 보간 메서드는 fillna 메서드에도 사용 가능
df = pd.DataFrame(np.random.randn(6, 3))

In [37]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

In [38]:
df

,0,1,2
0,-0.103423,0.217158,0.691543
1,0.225807,-0.020341,0.301206
2,0.230746,NaN,0.334497
3,-0.144216,NaN,1.314292
4,1.011699,NaN,NaN
5,0.194971,NaN,NaN


In [39]:
# 이전값으로 뒤 채우기
df.fillna(method='ffill')

,0,1,2
0,-0.103423,0.217158,0.691543
1,0.225807,-0.020341,0.301206
2,0.230746,-0.020341,0.334497
3,-0.144216,-0.020341,1.314292
4,1.011699,-0.020341,1.314292
5,0.194971,-0.020341,1.314292


In [40]:
# 채우기 개수 설정 가능
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.103423,0.217158,0.691543
1,0.225807,-0.020341,0.301206
2,0.230746,-0.020341,0.334497
3,-0.144216,-0.020341,1.314292
4,1.011699,NaN,1.314292
5,0.194971,NaN,1.314292


In [41]:
# fillna의 활용 가능한 경우는 많음. 예를 들어 Series의 평균값이나 중간값 전달 또한 가능
data = pd.Series([1., NA, 3.5, NA, 7])

In [42]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Part2. 데이터 변형 (필터링, 정제 및 다른 변형)

In [43]:
# 중복 제거하기: DataFrame에서 중복된 로우 처리
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [44]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [45]:
# DataFrame의 duplicated 메서드는 각 로우가 중복인지 아닌지 알려주는 불리언 Series 반환
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [46]:
# drop_duplicates 메서드는 duplicated 배열이 False인 DataFrame 반환
# 즉, 중복되지 않은 행만 반환
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [47]:
# duplicated와 drop_duplicates 메서드는 기본적으로 모든 컬럼에 적용
# 중복을 찾아내기 위한 부분합을 따로 지정할 수 있음
# 아래는 새로운 컬럼 추가 후 'k1' 컬럼에 기반하여 중복을 걸러내는 방법
data['v1'] = range(7)

In [48]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [49]:
# k1은 one컬럼과 two컬럼이 중복되므로, 가장 상단의 one/ two 컬럼만 추출 (이후 값들은 중복되어 나타난 값이므로!)
# 즉, 기본적으로 처음 발견된 값을 유지!
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [52]:
# 마지막으로 발견된 값을 반환하고 싶으면 keep='last' 옵션을 넘기면 됨
data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


In [53]:
data.drop_duplicates(['k1', 'k2']) # 얘는 중복된 5, 6행 중 5행만

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


In [54]:
data.drop_duplicates(['k1', 'k2'], keep='last') # 얘는 중복된 5, 6행 중 6행만

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 함수나 매핑 이용해 데이터 변형하기
* DataFrame의 컬럼이나 Series, 배열 내의 값을 기반으로 데이터 형태 변환하고 싶은 경우!
* 예로 가상으로 수집한 육류에 대한 정보를 변형해보자!

In [55]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 
                              'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [56]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [63]:
# 해당 육류가 어떤 동물의 고기인지 알려줄 수 있는 컬럼을 추가한다고 가정하자
# 육류별 동물을 담고 있는 사전 데이터
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

### MARK: Series의 map 메서드: 사전류의 객체나 어떤 함수를 인자로 받음
* map 메서드를 사용하면 데이터의 요소별 변환 및 데이터를 다듬는 작업을 편리하게 수용 가능

In [58]:
# 위 데이터는 육류 이름에 대소문자가 섞여 있는 사소한 문제 존재: str.lower 메서드를 사용하여 모두 소문자로 변경
lowercased = data['food'].str.lower()

In [59]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [65]:
data['animal'] = lowercased.map(meat_to_animal)

In [66]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [64]:
# 물론 함수를 넘겨서 동일한 작업을 수행할 수 있다
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### MARK: 값 치환하기
<pre> * fillna 메서드: 일반적인 값 치환 작업, 누락된 값 채우기.
 * map 메서드: 한 객체 안에서 값의 부분집합 변경
</pre>
* replace 메서드는 같은 작업에 대해 좀 더 간단하고 유연한 방법 제공

In [67]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [68]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [69]:
# -999는 누락된 데이터를 나타내기 위한 값
# replace()를 이용하면 이 값을 pandas에서 인식할 수 있는 NA값으로 치환한 Series를 새로 생성할 수 있음
# 주의사항: 인자로 inplace=True를 넘기지 않은 경우에만 가능 (inplace가 뭐길래,, 아까 바로 값 저장할 때 썼는데?)
# inplace=True: 원본 데이터를 바로 변경하겠다~ 고거지~
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [70]:
# 여러 개의 값을 한 번에 치환하려면? 값 대신 리스트 넘기기!
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [71]:
# 치환하려는 값마다 다른 값으로 치환하려면? 두 번째 인자(바꿀 값)도 리스트로!
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [72]:
# 각각의 리스트 대신 사전 이용 또한 가능
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [75]:
# 효진이의 실험 교실: 조건식이나 범위는 못 넣나?
data.replace(x>100, np.nan) # ㅎ.. x를 정의할 방법을 못 찾겠네

NameError: name 'x' is not defined

### MARK: 축 색인 이름 바꾸기
* Series의 값들처럼 축 이름 역시 유사한 방식으로 함수나 값을 이용해서 변환할 수 있다
* 새로운 자료구조를 만들지 않고 그 자리에서 바로 축 이름을 변경하는 것이 가능!

In [77]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [78]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [79]:
# Series와 마찬가지로 축 색인에도 map 메서드 존재
transform = lambda x: x[:4].upper()

In [80]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [82]:
# 대문자로 변경된 축 이름을 DataFrame의 index에 바로 대입할 수 있음 (기존 객체 변경)
data.index = data.index.map(transform)

In [83]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [84]:
# 기존 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메서드 사용
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [85]:
# 특히 rename 메서드는 사전 형식의 객체를 이용해서 축 이름 중 일부만 변경하는 것도 가능
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [87]:
# rename 메서드를 사용하면 DataFrame을 직접 복사해서 index와 columns 속성을 갱신할 필요 없이 바로 변경 가능
# 원본 데이터를 변경하려면 inplace=True 옵션을 넘겨주면 됨!
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [88]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [92]:
data.rename(index={'NEW': 'OLD'}, inplace=True)

In [93]:
data # ??? 왜 안 바뀌어?

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [94]:
data.rename(index={'NEW': 'OLD'})

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [95]:
data.index # 뛰어쓰기가.. 들어가있어... ! 아까 "New york"에서 4글자 잘라내서 그렇군!

Index(['INDIANA', 'COLO', 'NEW '], dtype='object')

In [96]:
data.rename(index={'NEW ': 'OLD'}, inplace=True)

In [97]:
data # 됐 다 리 ~ !

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
OLD,8,9,10,11


### MARK: 개별화와 양자화
* 연속성 데이터는 종종 개별로 분할하거나 분석을 위해 그룹별로 분리
* 수업에 참여하는 학생 그룹 데이터가 있고, 이를 나이대에 따라 분류한다고 가정하자

In [98]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [99]:
# 위 데이터를 pandas의 cut 함수를 이용해서 18-25/26-35/35-60/60 이상 그룹으로 나누어보자
bins = [18, 25, 35, 60, 100]

In [100]:
cats = pd.cut(ages, bins)

In [101]:
cats # 읭 19-24 같은데? 18은 포함 안 되는구만유?!

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [102]:
ages.append(18)

In [103]:
cats = pd.cut(ages, bins)

In [104]:
cats # 역시 18세는 포함이 안 돼서 NaN이 뜸니다 첫 값은 포함 안 하네!

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (60.0, 100.0], (35.0, 60.0], (35.0, 60.0], (25.0, 35.0], NaN]
Length: 13
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [105]:
ages.remove(18)

In [106]:
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [107]:
# pandas에서 cut 메서드 실행 후 반환하는 객체는 Categorical이라는 특수한 객체!
# Categorical은 그룹 이름이 담긴 배열!
# 이 객체는 codes 속성에 있는 ages 데이터에 대한 카테고리 이름을 categories라는 배열에 내부적으로 담고 있음
cats.codes

array([ 0,  0,  0,  1,  0,  0,  2,  1,  3,  2,  2,  1, -1], dtype=int8)

In [108]:
# 위 실행 결과는 18 포함한 값, 18은 범위 안에 속하지 않으므로 -1
# ages에서 18을 제외한 현 리스트를 기준으로 cut 실행하면 범위안에 모두 들어가있으므로 거뜬히 수행
cats = pd.cut(ages, bins)

In [109]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [110]:
# 간격을 나타내는 표기법은 소괄호로 시작해서 대괄호로 끝남
# 소괄호 쪽의 값은 포함하지 않고 대괄호 쪽의 값은 포함
# 즉, 컴퓨터 프로그래밍 배열에서 기본적인 인덱스를 논할 때 [0, n)라고 할 수 있음 (단, n은 배열의 길이)
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [113]:
# right=False를 넘겨 소괄호 대신 대괄호 쪽이 포함되지 않도록 바꿀 수 있음
# 이전 결과와 대괄호/소괄호가 반대됨을 알 수 있음!
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [112]:
# cats(pandas.cut 결과)에 대한 그룹 수
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [114]:
# labels 옵션으로 그룹의 이름을 직접 념겨줄 수 있음
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [115]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [116]:
# cut 함수에 명시적으로 그룹의 경곗값을 넘기지 않고 그룹의 개수를 넘겨주면
# 데이터의 최솟값과 최댓값을 기준으로 균등한 길이의 그룹을 자동으로 계산 (올ㅋ)
data = np.random.rand(20)

In [118]:
# precision 옵션은 소수점 아래 숫자를 제한할 때 사용 (아래와 같이 2인 경우, 소수점 아래 2자리까지!)
pd.cut(data, 4, precision=2)

[(0.73, 0.95], (0.73, 0.95], (0.049, 0.28], (0.73, 0.95], (0.049, 0.28], ..., (0.5, 0.73], (0.28, 0.5], (0.73, 0.95], (0.049, 0.28], (0.049, 0.28]]
Length: 20
Categories (4, interval[float64]): [(0.049, 0.28] < (0.28, 0.5] < (0.5, 0.73] < (0.73, 0.95]]

In [119]:
# 이럴 때 가장 적합한 함수로 qcut이 있는데 표본 변위치를 기반으로 데이터를 분리
# cut 함수를 사용하면 데이터의 분산에 따라 각각이 그룹마다 데이터 수가 다르게 나옴!
# qcut은 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있음
# 말이 어렵지만, 즉 정리하면 팀 나눌 때 동일한 팀원수 만들려면 qcut 함수가 유용하겠다~! 이것이지!
data = np.random.randn(1000) # 정규 분포

In [120]:
cats = pd.qcut(data, 4)

In [121]:
cats

[(0.684, 3.658], (0.684, 3.658], (-0.0125, 0.684], (0.684, 3.658], (-0.694, -0.0125], ..., (-0.0125, 0.684], (-2.932, -0.694], (-0.694, -0.0125], (-2.932, -0.694], (-0.694, -0.0125]]
Length: 1000
Categories (4, interval[float64]): [(-2.932, -0.694] < (-0.694, -0.0125] < (-0.0125, 0.684] < (0.684, 3.658]]

In [122]:
pd.value_counts(cats)

(0.684, 3.658]       250
(-0.0125, 0.684]     250
(-0.694, -0.0125]    250
(-2.932, -0.694]     250
dtype: int64

In [125]:
# 실수 값은 알아보기 힘드니까 정수값으로도 해볼까?
int_data = np.arange(16)

In [126]:
int_cats = pd.qcut(int_data, 4)

In [131]:
int_data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [127]:
int_cats

[(-0.001, 3.75], (-0.001, 3.75], (-0.001, 3.75], (-0.001, 3.75], (3.75, 7.5], ..., (7.5, 11.25], (11.25, 15.0], (11.25, 15.0], (11.25, 15.0], (11.25, 15.0]]
Length: 16
Categories (4, interval[float64]): [(-0.001, 3.75] < (3.75, 7.5] < (7.5, 11.25] < (11.25, 15.0]]

In [130]:
pd.value_counts(int_cats)

(11.25, 15.0]     4
(7.5, 11.25]      4
(3.75, 7.5]       4
(-0.001, 3.75]    4
dtype: int64

In [132]:
# cut 함수와 유사하게 변위치를 직접 지정해줄 수 있다 (변위치는 0부터 1까지)
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0125, 1.295], (1.295, 3.658], (-0.0125, 1.295], (-0.0125, 1.295], (-1.309, -0.0125], ..., (-0.0125, 1.295], (-1.309, -0.0125], (-1.309, -0.0125], (-1.309, -0.0125], (-1.309, -0.0125]]
Length: 1000
Categories (4, interval[float64]): [(-2.932, -1.309] < (-1.309, -0.0125] < (-0.0125, 1.295] < (1.295, 3.658]]

In [135]:
pd.qcut(int_data, [0, 0.1, 0.5, 0.9, 1.]) # 비율 지정! 0부터 10프로 이내, 10프로 초과 50프로 이내, ...

[(-0.001, 1.5], (-0.001, 1.5], (1.5, 7.5], (1.5, 7.5], (1.5, 7.5], ..., (7.5, 13.5], (7.5, 13.5], (7.5, 13.5], (13.5, 15.0], (13.5, 15.0]]
Length: 16
Categories (4, interval[float64]): [(-0.001, 1.5] < (1.5, 7.5] < (7.5, 13.5] < (13.5, 15.0]]

In [134]:
# cut 함수와 유사하게 변위치를 직접 지정해줄 수 있다 (변위치는 0부터 1까지)
# 그룹 분석과 변위치를 다룰 때에는 cut과 qcut 함수 같은 이산함수가 특히 더 유용
int_cats = pd.qcut(int_data, [0, 0.1, 0.5, 0.9, 1.])

In [136]:
pd.value_counts(int_cats)

(7.5, 13.5]      6
(1.5, 7.5]       6
(13.5, 15.0]     2
(-0.001, 1.5]    2
dtype: int64

### MARK: 특잇값 찾기 및 제외

In [137]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [138]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.004094,0.006535,-0.026269,0.011390
std,0.994985,0.981237,1.033715,1.003162
min,-3.152136,-3.034031,-3.241313,-2.839948
25%,-0.702792,-0.639199,-0.692678,-0.688907
50%,-0.012896,0.029756,-0.067074,-0.014240
75%,0.697999,0.650901,0.656927,0.682913
max,3.704037,3.555391,3.387045,3.572470


In [139]:
# 이 DataFrame의 한 컬럼에서 절댓값이 3을 초과하는 값 찾아내기
col = data[2]

In [140]:
col[np.abs(col) > 3]

89     3.007626
116    3.387045
386   -3.104948
404    3.010702
590    3.086891
742   -3.241313
Name: 2, dtype: float64

In [141]:
# 절댓값 3을 초과하는 값이 들어있는 모든 로우를 선택하려면 불리언 DataFrame에서 any 메서드 사용
data[(np.abs(data) > 3).any(1)] # any() 내부 인자는 의미하는 게 모야 0이나 2는 안 되네

,0,1,2,3
89,0.792327,0.111337,3.007626,0.014854
116,-0.974964,-0.468348,3.387045,-1.100515
210,3.658015,0.931028,0.051565,-0.215346
322,-3.152136,-0.134635,0.589890,1.022147
386,-0.315036,0.322826,-3.104948,-0.203527
404,-0.033644,0.590664,3.010702,0.327015
506,3.704037,-0.346383,-0.019303,-0.227951
590,0.411227,0.757374,3.086891,-0.577705
637,-0.820616,3.555391,1.227549,-1.737694
682,-0.244732,-0.311780,-0.012453,3.572470


In [145]:
# 아래 코드로 -3이나 3을 초과하는 값을 -3또는 3으로 지정할 수 있다 (?)
# 3을 초과하면 3으로 바꾸고 -3보다 작으면 -3으로 바꾸는 것!
data[np.abs(data) > 3] = np.sign(data) * 3

In [146]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.005304,0.005623,-0.026415,0.010817
std,0.989917,0.978004,1.031162,1.001289
min,-3.000000,-3.000000,-3.000000,-2.839948
25%,-0.702792,-0.639199,-0.692678,-0.688907
50%,-0.012896,0.029756,-0.067074,-0.014240
75%,0.697999,0.650901,0.656927,0.682913
max,3.000000,3.000000,3.000000,3.000000


In [147]:
# np.sign(data)는 data 값이 양수인지 음수인지에 따라 1이나 -1이 담긴 배열 반환
np.sign(data)

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,1.0,1.0
3,1.0,1.0,-1.0,1.0
4,1.0,-1.0,-1.0,1.0
...,...,...,...,...
995,-1.0,-1.0,-1.0,1.0
996,1.0,1.0,-1.0,-1.0
997,1.0,1.0,1.0,1.0
998,1.0,1.0,1.0,-1.0


In [148]:
np.sign(data) * 3

,0,1,2,3
0,3.0,-3.0,3.0,-3.0
1,3.0,-3.0,-3.0,-3.0
2,3.0,-3.0,3.0,3.0
3,3.0,3.0,-3.0,3.0
4,3.0,-3.0,-3.0,3.0
...,...,...,...,...
995,-3.0,-3.0,-3.0,3.0
996,3.0,3.0,-3.0,-3.0
997,3.0,3.0,3.0,3.0
998,3.0,3.0,3.0,-3.0


In [149]:
data

,0,1,2,3
0,1.216060,-0.169956,0.378506,-0.813198
1,0.762590,-1.737011,-1.126491,-0.026817
2,1.572870,-0.105293,0.852785,0.981230
3,0.356046,0.136542,-2.000381,2.078730
4,1.024836,-2.219215,-0.395451,1.094486
...,...,...,...,...
995,-0.978163,-0.974201,-0.507152,0.187411
996,0.125527,0.420523,-0.995259,-1.248315
997,1.033808,1.210398,0.457096,0.546346
998,0.030789,0.801582,1.432564,-1.983588


### MARK: 치환과 임의 샘플링
* numpy.random.permutation 함수 이용: Series나 DataFrame의 로우를 쉽게 임의 순서로 재배치
* 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수로 넘기면 바뀐 순서가 담긴 정수 배열 생성

In [150]:
df = pd.DataFrame(np.arange(5*4).reshape(5, 4))

In [151]:
sampler = np.random.permutation(5)

In [152]:
sampler

array([2, 1, 0, 4, 3])

In [153]:
# 이 배열은 iloc 기반의 색인이나 take 함수에서 사용 가능
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [154]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


In [155]:
# 치환 없이 일부만 임의로 선택하려면 sample 메서드 활용
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15


In [156]:
df.sample(n=3) # 할 때 마다 임의의 값은 변경~!

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11


In [157]:
# 표본을 치환을 통해 생성해내려면 sample에 replace=True 옵션 넘기기 (반복 선택 허용)
choices = pd.Series([5, 7, -1, 6, 4])

In [159]:
draws=choices.sample(n=10, replace=True)

In [160]:
draws # 반복 선택을 허용한다긴 했는데, 선택이 아애 안 될 수도 있구나!

0    5
4    4
3    6
4    4
3    6
4    4
0    5
0    5
3    6
2   -1
dtype: int64

### MARK: 표시자/더미 변수 계산 (get_dummies)
* 분류값을 '더미'나 '표시자' 행렬로 전환하는 것
* 어떤 DataFrame의 한 컬럼에 k가지의 값이 있다면
  * k개의 컬럼이 있는 DataFrame이나 행렬을 만들고 값으로는 1과 0을 채워넣는 것

In [161]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

In [170]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [162]:
# 위의 사전을 위치를 index, 요소를 column으로 하여 1과 0으로 표시한 것!
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [163]:
# 표시자 DataFrame 안에 있는 컬럼을 접두어를 추가한 후 다른 데이터와 병합하고 싶을 경우
# get_dummies 함수의 prefix 인자 사용
dummies = pd.get_dummies(df['key'], prefix='key')

In [164]:
df_with_dummy = df[['data1']].join(dummies)

In [165]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [166]:
dummies2 = pd.get_dummies(df['key'], prefix='I am')

In [172]:
df_with_dummy2 = df[['key']].join(dummies2)

In [173]:
df_with_dummy2

,key,I am_a,I am_b,I am_c
0,b,0,1,0
1,b,0,1,0
2,a,1,0,0
3,c,0,0,1
4,a,1,0,0
5,b,0,1,0


In [174]:
# DataFrame의 한 로우가 여러 카테고리에 속한다면 조금 복잡해짐
mnames = ['movie_id', 'title', 'genres']

In [176]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)

/Users/adorable_hj/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [177]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [178]:
# 표시자 값을 추가하려면 데이터 묶음에서 "유일한" 장르 목록을 추출
all_genres = []

In [179]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [180]:
genres = pd.unique(all_genres)

In [181]:
# 중복이 없는 장르 목록
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [195]:
# movies.genres와는 다름!! 이거는 모든 영화별 장르!
movies.genres

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genres, Length: 3883, dtype: object

In [182]:
# 표시자 DataFrame을 생성하기 위해 0으로 초기화된 DataFrame 생성
zero_matrix = np.zeros((len(movies), len(genres)))

In [183]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [196]:
# 각 영화를 순회하면서 각 로우의 항목을 1로 설정
dummies # 장르별 맞다/아니다 체크를 한 거쥐

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
gen = movies.genres[0]

In [185]:
gen

"Animation|Children's|Comedy"

In [186]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [187]:
# 각 장르의 컬럼 색인을 계산하기 위해 dummies.columns 사용
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [188]:
movies.genres

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genres, Length: 3883, dtype: object

In [190]:
# iloc을 이용해서 색인에 맞게 값 대임
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [193]:
# movies와 조합
# 앞에 키워드 붙여주기 ex) Genre_Animation
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [194]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [197]:
# get_dummies와 cut 같은 이산 함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용 가능
# 예제 값이 불변하도록 seed 함수를 이용하여 난수 시드값 지정
np.random.seed(12345)

In [198]:
values = np.random.rand(10)

In [199]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [200]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [201]:
# get_dummies 어려워도 일단 넘어가기! 나중에 또 살펴봄!
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### Part3: 문자열 다루기
* 대부분의 텍스트 연산은 문자열 객체의 내장 메서드로 간단히 처리
* 좀 더 복잡한 패턴 매칭이나 텍스트 조작은 정규 표현식 필요로 함
* pandas는 배열 데이터 전체에 쉽게 정규 표현식을 적용하고 누락된 데이터를 편리하게 처리할 수 있는 기능 포함

### MARK: 문자열 객체 메서드
* 문자열을 다뤄야 하는 대부분의 애플리케이션은 내장 문자열 메서드만으로 충분

In [202]:
val = 'a,b, guido'

In [203]:
val.split(',') # 공백은 제거 안 됨

['a', 'b', ' guido']

In [204]:
# 공백 문자(줄바꿈 포함)를 제거하는 strip 메서드와 조합해서 사용
pieces = [x.strip() for x in val.split(',')]

In [205]:
pieces

['a', 'b', 'guido']

In [206]:
# 분리된 문자열은 더하기 연산을 이용해서 사용자가 원하는 문자열과 합칠 수 있음
first, second, third = pieces

In [207]:
first + '::' + second + '::' + third

'a::b::guido'

In [208]:
# 위는 범용적 방법이 아님. 파이썬스러운 방법은 join 메서드로 합치려는 문자 전달
'::'.join(pieces)

'a::b::guido'

In [209]:
# 일치하는 부분문자열 위치 찾는 방법
# (1) index, find 사용 (2) in 예약어 사용
# in 예약어는 일치하는 부분 문자열을 쉽게 찾을 수 있음
'guido' in val

True

In [210]:
val.index(',') # 존재하지 않을 경우 예외 발생

1

In [211]:
val.find(':') # 존재하지 않기 때문에 -1 반환

-1

In [212]:
# count는 특정 부분 문자열이 몇 건 발견되었는 지 반환
val.count(',')

2

In [213]:
# replace는 찾아낸 패턴을 다른 문자열로 치환
# 대체할 문자열로 비어있는 패턴을 삭제하기 위한 방법으로도 사용
val.replace(',', '::')

'a::b:: guido'

In [214]:
# 얘는 사전을 인자로 못 넘기나?
val.replace(' ', '')

'a,b,guido'

In [215]:
# 안 되 네 ㅎ 역시 갓다스만 되는 것인가,,
val.replace({' ': '', ',':'::'})

TypeError: replace() takes at least 2 arguments (1 given)

### 정규 표현식
* 문자열 패턴을 찾는 유연한 방법 제공
* 흔히 regex라 불리는 단일 표현식은 정규 표현 언어로 구성된 문자열
* 파이썬에는 re 모듈이 내장되어 있어 문자열에 대한 정규 표현식 처리
<pre> <b>re 모듈 함수</b>
 : 패턴 매칭, 치환, 분리 세가지로 나눌 수 있음. 세 가지는 모두 서로 연관.
</pre>
* 정규 표현식은 텍스트 내에 존재하는 패턴을 표현, 이를 여러 가지 다양한 목적으로 사용할 수 있도록 되어 있음
* 여러 공백 문자가 포함된 문자열을 나눌 때 하나 이상의 공백 문자를 의미하는 '\s+'를 사용하여 문자열 분리
  * 공백 문자는 [탭/스페이스/개행문자] 가 포함됨


In [216]:
import re

In [219]:
text = "foo bal\t baz \tqux"

In [220]:
text

'foo bal\t baz \tqux'

In [221]:
# 먼저 정규 표현식이 컴파일되고 그다음에 split 메서드가 실행
re.split('\s+', text)

['foo', 'bal', 'baz', 'qux']

In [223]:
# re.compile로 직접 정규 표현식을 컴파일 한 후 얻은 표현식 객체를 재사용하는 것도 가능
# 같은 정규 표현식을 다른 문자열에도 적용해야 한다면 re.compile으로 정규 표현식 객체를 만들어 쓰는 방법을 추천!
# 그러면 CPU 사용량을 아낄 수 있음
regex = re.compile('\s+')

In [224]:
regex.split(text)

['foo', 'bal', 'baz', 'qux']

In [225]:
# 정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드 사용
# 정규 표현식에서 \ 문자가 이스케이프되는 문제를 피하려면 raw 문자열 표기법 사용!
# 그러면 \를 일반 문자로 처리하기 때문에 간단히 표현 가능, 즉 'C:\\x' 대신 'C:\x'를 사용
regex.findall(text)

[' ', '\t ', ' \t']

#### match와 search는 findall 메서드와 관련
* findall(): 문자열에서 일치하는 모든 부분문자열 찾아줌
* search(): 패턴과 일치하는 첫 번째 존재 반환
* match(): 엄격! 문자열의 시작부분에서 일치하는 것만 찾아줌

In [230]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [227]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # ~~~@~~.~~ 형식 값 찾기

In [228]:
# re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 함
regex = re.compile(pattern, flags=re.IGNORECASE)

In [231]:
# findall 메서드 사용하여 이메일 주소 리스트 생성
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [232]:
# search는 텍스트에서 첫번째 이메일 주소만 찾음! (match 객체 반환)
# match 객체는 그 정규 표현 패턴이 문자열 내에서 위치하는 시작점과 끝점만 알려줌
m = regex.search(text)

In [233]:
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [234]:
text[m.start():m.end()]

'dave@google.com'

In [235]:
# regex.matchs는 None을 반환
# 왜? 그 정규 표현 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문
print(regex.match(text))

None


In [236]:
# sub(): 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열 변환
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [237]:
# 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사
# 세가지 컴포넌트로 나눠야 한다면 각 패턴을 괄호로 묶어준다
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)

In [239]:
m = regex.match('wesm@bright.net')

In [240]:
m.groups() # 튜플 형태로 저장

('wesm', 'bright', 'net')

In [241]:
# findall(): 패턴에 그룹이 존재한다면 튜플의 목록(리스트)를 반환
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [242]:
# sub() 역시 특수한 기호를 이용해서 각 패턴 그룹에 접근할 수 있음
# \1, \2 등으로 표현하며, \1은 첫번째, \2는 두번째로 찾은 그룹을 의미
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### pandas의 벡터화된 문자열 함수

In [244]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com',
        'Wes': np.nan} # 문자열을 담고 있는 컬럼에 누락된 값이 있다?!

In [247]:
data = pd.Series(data)

In [248]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [249]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [ ]:
# 문자열과 정규표현식은 data.map을 사용해서 각 값에 적용 가능
# 단, NA 값을 만나면 실패!
# Series에는 NA값을 건너뛰도록 하는 간결한 문자열 처리 메서드 존재 (Series의 str 속성 이용)

In [251]:
# 각 이메일 주소가 'gmail'을 포함하고 있는지 str.contains를 이용하여 검사할 수 있음
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [253]:
# 정규표현식을 IGNORECASE 같은 re 옵션과 함께 사용하는 것도 가능
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [254]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [255]:
# 벡터화된 요소 꺼내오는 방법: str.get 이용하거나 str 속성의 색인 이용
matches=data.str.match(pattern, flags=re.IGNORECASE)

In [256]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [259]:
# 내제된 리스트 원소에 접근하기 위해서는 색인 넘기기
matches.str.get(1) # matches는 문자열을 값을 갖고 있지 않아서 str 을 접근이 안된다는데용,,

AttributeError: Can only use .str accessor with string values!

In [260]:
matches.str[0] # 마 찬 가 지 NaN 값이 안 나오고 에러가 떠버리넹

AttributeError: Can only use .str accessor with string values!

In [261]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object